<a href="https://colab.research.google.com/github/sabrina-morais/IA/blob/main/Trab01_RegLinear_Lact_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -U dtale

In [ ]:
import pandas as pd

import numpy as np
import matplotlib.pyplot as plt

from google.colab import drive

drive.mount('/content/drive')
folder = '/content/drive/My Drive/Colab Notebooks/Trab01_RegLinear_Lact/'
df = pd.read_csv(folder + 'dataset.csv')
df.head()
#df.describe()


In [ ]:
import dtale
import dtale.app as dtale_app
dtale_app.USE_COLAB  = True
#dtale_app.USE_NGROK = True

import seaborn as sns #visualizador de dados estatísticos

In [ ]:
dtale.show(df)

1. Análise inicial dos dados

 a. Construir uma tabela com as seguintes informações:

- Animal | Prod. Mínima - 1º Lact. | Prod. média - 1º Lact. | Prod. máxima - 1º Lact. | Prod. Mínima - 2º Lact. | Prod. média - 2º Lact. | Prod. máxima - 2º Lact. | Prod. Mínima - 3º+ Lact. | Prod. média - 3º+ Lact. | Prod. máxima - 3º+ Lact.

In [ ]:
result = df.groupby(['NUMERO', 'lactation'])['PRODUCAO_PESO'].agg(['min', 'max', 'mean'])
print(result)
result.to_csv(folder + 'result.csv')

In [ ]:
df_1502=df.query('NUMERO ==1502 & lactation ==1')
df_1502.describe()

b. Realizar um ajuste linear (y = ax + b) via código (sem uso de bibliotecas) para as curvas de cada animal (por lactação), informando:

- Animal | lactação | a | b | erro quadrático médio


Exemplo sala

y = ax + b
a: coeficiente angular
b: termo independente

dados = [[1,2], [2,5], [2,7], [3,6], [4,3], [4,8], [5,9]]

a = 0.5
b= 1

y = ax + b
y = ax + b --> 0.5*1+1
erro1 = 2-1.5 = 0.5 (a: int, b:int) ->int

melhor_a = 0
melhor_b = 0
menor_erro = 999999999

for i in range(10):
    a = random.randint(-5,5)
    b = random.randint(-50,50)


    total_erro = 0
    for d in dados:
        h = a*d[0]+b
        #erro = d[1]-h
        #erro = np.power (d[1]-h,2)
        erro = np.absolute (d[1]-h)
        total_erro = total_erro + erro
    if(total_erro < menor_erro):
        menor_erro = total_erro
        melhor_a = a
        melhor_b = b
    print(total_erro/len(dados))





In [ ]:
def calcular_ajuste_linear(x, y):

    # Calcula os parâmetros da reta (a e b) e o erro quadrático médio (EQM) para ajuste linear.
    n = len(x)

    # Calcular os coeficientes da reta
    x_mean = sum(x) / n
    y_mean = sum(y) / n

    numerator_a = sum((x[i] - x_mean) * (y[i] - y_mean) for i in range(n))
    denominator_a = sum((x[i] - x_mean) ** 2 for i in range(n))

    # Lidar com denominador zero
    if denominator_a == 0:
        return None, None, float('inf')  # Indica

    a = numerator_a / denominator_a
    b = y_mean - a * x_mean

    # Calcular o erro quadrático médio
    y_pred = [a * xi + b for xi in x]
    eqm = sum((y[i] - y_pred[i]) ** 2 for i in range(n)) / n

    return a, b, eqm

def ajustar_curvas(df):
    resultados = []

    for (animal, lactacao), grupo in df.groupby(['NUMERO', 'lactation']):
        x = grupo['dim'].tolist()
        y = grupo['PRODUCAO_PESO'].tolist()

        a, b, eqm = calcular_ajuste_linear(x, y)

        resultados.append({
            'Animal': animal,
            'Lactação': lactacao,
            'a': a,
            'b': b,
            'Erro Quadrático Médio': eqm
        })

    return pd.DataFrame(resultados)

# Aplicar a função e obter os resultados
resultados_df = ajustar_curvas(df)

print(resultados_df)


In [ ]:
# imprimir os resultados
plt.figure(figsize=(10, 6))

for index, row in resultados_df.iterrows():
    animal = row['Animal']
    lactacao = row['Lactação']
    a = row['a']
    b = row['b']

    grupo = df.query(f'NUMERO == {animal} & lactation == {lactacao}')
    x = grupo['dim']
    y = grupo['PRODUCAO_PESO']

    plt.scatter(x, y, label=f'Animal {animal}, Lactação {lactacao}')
    plt.plot(x, a * x + b, linestyle='--')  # traça a linha de regressão

plt.xlabel('Days in milk')
plt.ylabel('Milk production (kg)')
plt.title('Ajuste Linear por Animal e Lactação')
#plt.legend()
plt.grid(True)
plt.show()

c. Verificar o desempenho da curva de cada animal quando aplicada para estimar a produção de outros animais

- Animal Utilizado | Animal (lactação) | Animal Alvo | Lactação Alvo | Erro quadrático médio

In [ ]:
def calcular_erro_entre_animais(df, resultados_df):
   #Calcula o erro quadrático médio ao usar a curva de um animal para estimar a produção de outro.

    resultados_erro = []

    for _, row_ajuste in resultados_df.iterrows():
        animal_utilizado = row_ajuste['Animal']
        lactacao_utilizada = row_ajuste['Lactação']
        a = row_ajuste['a']
        b = row_ajuste['b']

        for animal_alvo in df['NUMERO'].unique():
            for lactacao_alvo in df.query(f'NUMERO == {animal_alvo}')['lactation'].unique():
                # Skip if it's the same animal and lactation used for fitting
                if animal_alvo == animal_utilizado and lactacao_alvo == lactacao_utilizada:
                    continue

                grupo_alvo = df.query(f'NUMERO == {animal_alvo} & lactation == {lactacao_alvo}')
                x_alvo = grupo_alvo['dim'].tolist()
                y_alvo = grupo_alvo['PRODUCAO_PESO'].tolist()

                # Predict using the curve of the "used" animal
                y_pred = [a * xi + b for xi in x_alvo]

                # Calculate mean squared error
                eqm = sum((y_alvo[i] - y_pred[i]) ** 2 for i in range(len(y_alvo))) / len(y_alvo)

                resultados_erro.append({
                    'Animal Utilizado': animal_utilizado,
                    'Lactação Utilizada': lactacao_utilizada,
                    'Animal Alvo': animal_alvo,
                    'Lactação Alvo': lactacao_alvo,
                    'Erro Quadrático Médio': eqm
                })

    return pd.DataFrame(resultados_erro)

# Aplicar as funções e obter os resultados
resultados_df = ajustar_curvas(df)
resultados_erro_df = calcular_erro_entre_animais(df, resultados_df)

print(resultados_erro_df)
resultados_erro_df.to_csv(folder + 'resultados_erro_df.csv')

d. Quais animais tiveram menor erro médio:

- Em seus ajustes de curva da primeira lactação quando aplicada para predizer a produção de outros animais

- Em seus ajustes de curva da segunda lactação quando aplicada para predizer a produção de outros animais

- Em seus ajustes de curva da terceira lactação quando aplicada para predizer a produção de outros animais

In [ ]:
# Animais com menor erro médio para cada lactação
for lactacao in [1, 2, 3]:
    erro_medio_por_animal = resultados_erro_df[resultados_erro_df['Lactação Utilizada'] == lactacao].groupby('Animal Utilizado')['Erro Quadrático Médio'].mean()
    melhor_animal = erro_medio_por_animal.idxmin()
    menor_erro = erro_medio_por_animal.min()

    print(f"Para a lactação {lactacao}, o animal com menor erro médio foi o {melhor_animal} com um erro de {menor_erro:.2f}")

In [ ]:
df_1584_1=df.query('NUMERO ==1584 & lactation ==1')
df_1512_2=df.query('NUMERO ==1512 & lactation ==2')
df_1512_3=df.query('NUMERO ==1512 & lactation ==3')

In [ ]:
plt.xlabel('Days in milk')
plt.ylabel('Milk production (kg)')
plt.title('Animal com menor erro médio')

plt.scatter(df_1584_1['dim'], df_1584_1['PRODUCAO_PESO'], color = 'red', marker='*', s=50)
plt.scatter(df_1512_2['dim'], df_1512_2['PRODUCAO_PESO'], color = 'blue', marker='+', s=50)
plt.scatter(df_1512_3['dim'], df_1512_3['PRODUCAO_PESO'], color = 'green', marker='*', s=50)

plt.grid(True)
plt.show()

2. Como você analisa os dados do exercício em relação a capacidade de generalização deste modelo de ajuste linear aplicado?

O modelo de ajuste linear não é satisfatório para generalizar os dados de lactação por serem distribuidos uma curva convexa, onde os pontos distantes do ponto de intersecção com a linha de ajuste apresentam um erro médio quadrático alto.

In [ ]:
# Modelo filtro e impressão
df1=df.query('lactation ==1')
df1.describe()
df2=df.query('lactation ==2')
#df2.describe()
df3=df.query('lactation ==3')
#df3.describe()
df4=df.query('lactation ==4')
#df4.describe
df5=df.query('lactation ==5')
#df5.describe

#df1['PRODUCAO_PESO'].hist()
#df1['PRODUCAO_PESO'].boxplot()
#df1['PRODUCAO_PESO'].plot.box()

plt.xlabel('Days in milk')
plt.ylabel('Milk production (kg)')
plt.title('Produções por Lactação')

plt.scatter(df1['dim'], df1['PRODUCAO_PESO'], color = 'red', marker='*', s=50)
plt.scatter(df2['dim'], df2['PRODUCAO_PESO'], color = 'blue', marker='+', s=50)
plt.scatter(df3['dim'], df3['PRODUCAO_PESO'], color = 'green', marker='*', s=50)
plt.scatter(df4['dim'], df4['PRODUCAO_PESO'], color = 'orange', marker='+', s=50)
plt.scatter(df5['dim'], df5['PRODUCAO_PESO'], color = 'black', marker='+', s=50)

In [ ]:
# Modelo filtro e impressão

df_1502_1=df.query('NUMERO ==1502 & lactation ==1')
df_1502_2=df.query('NUMERO ==1502 & lactation ==2')
df_1502_3=df.query('NUMERO ==1502 & lactation ==3')
df_1502_4=df.query('NUMERO ==1502 & lactation ==4')
df_1502_5=df.query('NUMERO ==1502 & lactation ==5')

plt.xlabel('Days in milk')
plt.ylabel('Milk production (kg)')
plt.title('Desempenho do animal 1502')

plt.scatter(df_1502_1['dim'], df_1502_1['PRODUCAO_PESO'], color = 'red', marker='*', s=50)
plt.scatter(df_1502_2['dim'], df_1502_2['PRODUCAO_PESO'], color = 'blue', marker='+', s=50)
plt.scatter(df_1502_3['dim'], df_1502_3['PRODUCAO_PESO'], color = 'green', marker='*', s=50)
plt.scatter(df_1502_4['dim'], df_1502_4['PRODUCAO_PESO'], color = 'orange', marker='+', s=50)
plt.scatter(df_1502_5['dim'], df_1502_5['PRODUCAO_PESO'], color = 'black', marker='+', s=50)